In [ ]:
#9章　データの取得

#9.1

import sys, re

regex = sys.argv[1]

for line in sys.stdin:
    if re.search(regex, line):
        sys.stdout.write(line)

In [ ]:
#9.2

#重要：csvファイルを扱う際には、rかwの後ろにaを置いて、常にバイナリモードでopenするべき。

import csv

def process(d, s, c):
    print(d, s, c)

with open('tab_delimited_stock_prices.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        date = row[0]
        symbol = row[1]
        closing_price = float(row[2])
        process(date, symbol, closing_price)

In [ ]:
with open('colon_delimited_stock_prices.txt','r') as f:
    reader = csv.DictReader(f, delimiter=':')
    for row in reader:
        date = row['date']
        symbol = row['symbol']
        closing_price = float(row['closing_price'])
        process(date, symbol, closing_price)

In [ ]:
today_prices = { 'AAPL':90.91, 'MSFT':41.68, 'FB':64.5 }

with open('comma_delimited_stock_prices.txt','w') as f:
    writer = csv.writer(f, delimiter=',')
    for stock, price in today_prices.items():
        writer.writerow([stock, price])
        
# AAPL,90.91
# MSFT,41.68
# FB,64.5

In [ ]:
#9.3

#　BeautifulSoup()にHTML文書を渡す。
from bs4 import BeautifulSoup
import requests
html = requests.get("http://www.example.com").text
soup = BeautifulSoup(html, 'html5lib')

print("==== #元のHTML ====\n\n", html)

#　最初の<p>タグを探し出す
first_paragraph = soup.find('p')
print("\n==== #最初のpタグ内 ====\n\n", first_paragraph)

#　textプロパティを通して、タグの文字列部分を取り出す。
first_paragraph_text = soup.p.text
print("\n==== #最初の一文 ====\n\n", first_paragraph_text)

first_paragraph_words = soup.p.text.split()
print("\n==== #最初の単語 ====\n\n", first_paragraph_words)

#　Tagオブジェクトを辞書として使えば、属性にアクセスできる。
#first_paragraph_id = soup.p['id'] #idが無ければKeyError
first_paragraph_id2 = soup.p.text.split('id') #idが無ければNoneが返る
print("\n==== #属性にアクセス ====\n\n", first_paragraph_id2)

#　複数のタグを一度に取り出す。
all_paragraphs = soup.find_all('p')
paragraphs_with_ids = [p for p in soup('p') if p.get('id')]
print("\n==== #複数のタグを一度に取り出す ====\n\n", all_paragraphs)

#　注意：複数の<div>の中に同じ<span>があれば、それらをその都度返す
#　その様な場合は、さらに工夫が必要
spans_inside_divs = [span
                    for div in soup('div') #ページ上の<div>要素ごとに繰り返し
                    for span in div('span')] #その中の<span>要素で繰り返し

print("==== #複数の<div>の中の<span>を取り出す ====", spans_inside_divs)

In [ ]:
#9.4.1

import json
serialized = """{ "title":"Data Science Book", "author":"Joel Grus", "publicationYear":2014, "topics":["data","science","data science"]}"""

#　jsonからPython辞書を作る
deserialized = json.loads(serialized)
if "data science" in deserialized["topics"]:
    print(deserialized)

In [ ]:
#　9.4.2

import requests, json
endpoint = "https://api.github.com/users/joelgrus/repos"
repos = json.loads(requests.get(endpoint).text)

print(repos)

In [ ]:
from dateutil.parser import parse
from collections import Counter

dates = [parse(repo["created_at"]) for repo in repos]
print("==== dates ====\n\n", dates)

month_counts = Counter(date.month for date in dates)
print("\n==== month ====\n\n", month_counts)

weekday_counts = Counter(date.weekday() for date in dates)
print("\n==== weekday ====\n\n", weekday_counts)

last_5_repositories = sorted(repos,
                            key=lambda r: r["created_at"],
                            reverse=True)[:5]
print("\n==== last_5_repositories ====\n\n", last_5_repositories)

last_5_languages = [repo["language"]
                   for repo in last_5_repositories]
print("\n==== last_5_languages ====\n\n", last_5_languages)

In [ ]:
#9.4.3

#Rotten Tomatoes
#https://www.rottentomatoes.com/

#Klout
#https://klout.com/home

#Yelp
#https://www.yelp.com/

#IMDb
#http://www.imdb.com/

#Python API
#http://pythonapi.com/

#Python for Beginners API list
#http://www.pythonforbeginners.com/development/list-of-python-apis/

#Programmable Web
#https://www.programmableweb.com/

In [ ]:
#9.5 Twitter API

#　事前にtwythonをインストールする
#　conda install -c conda-forge twython 

#　事前にtwitterAPIを取得する。電話番号を登録してあるTwitterアカウントが必要
#　https://apps.twitter.com/

from twython import Twython

#　キー関連は後で削除すること！！！！！！

CONSUMER_KEY = '*****'
CONSUMER_SECRET = '*****'
ACCESS_TOKEN = '*****'
ACCESS_TOKEN_SECRET = '*****'

twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET)

# "data science"を含むツイートを検索する
for status in twitter.search(q='"data science"')["statuses"]:
    user = status["user"]["screen_name"].encode('utf-8')
    text = status["text"].encode('utf-8')
    print(user,':',text)
    print()

In [ ]:
from twython import TwythonStreamer

#　データを大域変数に格納するのは稚拙な手法ではあるが、サンプルコードを簡単にできる

tweets = []

class MyStreamer(TwythonStreamer):
    """streamとやりとりを行う方法を定義する、TwythonStreamerのサブクラス"""
    
    def on_success(self, data):
        """Twitterがデータを送ってきたらどうするか？
        ここでdataは、ツイートを表すPython辞書として渡される"""
        
        #　英語のツイートのみを対象とする
        if data['lang'] is 'en':
            tweets.append(data)
            print("received tweet #", len(tweets))
            
        #　十分なツイートが得られたら終了
        if len(tweets) >= 10:
            self.disconnect()
            
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#　公開されているツイート（statuses）からキーワード'data'を持つものを収穫する
stream.statuses.filter(track='data')

#　すべてのツイート（statuses）の中からランダムに収穫を行うには、次のコードを使う
#stream.statuses.sample()

top_hashtags = Counter(hashtag['text'].lower()
                      for tweet in tweets
                      for hashtag in tweet["entities"]["hashtags"])

print(top_hashtags.most_common(5))